In [ ]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir data

In [ ]:
cd data

/content/data


In [ ]:
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/grapes.npy
!wget https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy

--2022-12-29 15:55:54--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 173.194.197.128, 173.194.198.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113462128 (108M) [application/octet-stream]
Saving to: ‘apple.npy’

apple.npy           100%[===================>] 108.21M   166MB/s    in 0.7s    

2022-12-29 15:55:55 (166 MB/s) - ‘apple.npy’ saved [113462128/113462128]

--2022-12-29 15:55:55--  https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pineapple.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 173.194.197.128, 173.194.198.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98055744 (94M) [application/octet

In [ ]:
!ls

apple.npy  banana.npy  grapes.npy  pineapple.npy


In [ ]:
cd

/root


In [ ]:
from sklearn.model_selection import train_test_split as train_test_split
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils import np_utils
from random import randint
import numpy as np
import os
from PIL import Image

In [ ]:
#define some constants
#total no of fruits
N_FRUITS = 4
#create a dictionary with the fruits
FRUITS = {0:"Apple", 1:"Banana",2:"Grape",3:"Pineapple"}
#number of samples to take in each class
N = 5000
#some other constants
N_EPOCHS = 10

#data files in the same order as defined in FRUITS dictionary
files = ["apple.npy","banana.npy","grapes.npy", "pineapple.npy"]

In [ ]:
def load(dir,reshaped, files):
  #take the .npy files and return them as arrays.
  data = [ ]
  for file in files:
    f = np.load(dir + file)
    if reshaped:
      new_f =[]
      for i in range(len(f)):
        x = np.reshape(f[i], (28,28))
        x = np.expand_dims(x, axis = 0)
        x = np.reshape(f[i], (28,28,1))
        new_f.append(x)
      f = new_f
    data.append(f)
  return data

def normalize(data):
  #takes a list ot a list of lists and returns its normalized form
  return np.interp(data, [0,255],[-1,1])

def denormalize(data):
  #takes a list ot a list of lists and returns its denormalized form 
  return np.interp(data,[-1,1],[0,255])

def visualize(array):
  #visualize a 2D array as an Image
  array = np.reshape(array, (28,28))
  img = Image.fromarray(array)
  return img 

def set_limit(arrays, n):
  #limit elements from each array up to n element and return a single list
  new = []
  for array in arrays:
    i = 0
    for item in array:
      if i == n:
        break
      new.append(item)
      i += 1
  return new

def make_labels(N1, N2):
  #make labels from 0 to N1, each repeated N2 times
  labels = []
  for i in range(N1):
    labels += [i] *N2
  return labels

In [ ]:
fruits = load("/content/data/", False, ["apple.npy"])

In [ ]:
visualize(fruits[0][0])

In [ ]:
#second argument is True for reshaping the image to 28 x 28 form. A conv net expects this format
fruits = load("/content/data/", True, files )

fruits = set_limit(fruits, N)
#to make the values -1 to 1. This is because neural network works better 
fruits = list(map(normalize, fruits))

labels = make_labels(N_FRUITS, N)
# 5 percent of the dataset will be reserve for training purposes only. (test_size)
x_train, x_test, y_train, y_test = train_test_split(fruits, labels, test_size=0.05)

#one hot encoding
Y_train = np_utils.to_categorical(y_train, N_FRUITS)
Y_test = np_utils.to_categorical(y_test, N_FRUITS)

In [ ]:
#convu neural network
model = Sequential()
# the input shape is 28 by 28 because of the image size. 1 is due to it being black and white image
model.add(Conv2D(32, kernel_size = (3 , 3), activation = "relu", input_shape = (28,28,1)))
model.add(Conv2D(64, (3,3), activation = "relu"))
#resizing kind of error. simplified image.
model.add(MaxPooling2D(pool_size = (2,2)))
#Dropout layout ensures that we are not overfitting the model
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
#the N_FRUITS is because we want the number of nodes to be the same as the no of catergory in N_FRUITS
model.add(Dense(N_FRUITS, activation = "softmax"))


In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"] )
model.fit(np.array(x_train), np.array(Y_train), batch_size = 32, epochs = N_EPOCHS)
print("Training complete")
print("Evaluating model")
preds = model.predict(np.array(x_test))

score = 0
for i in range(len(preds)):
  if np.argmax(preds[i]) == y_test[i]:
    score += 1
print ("Accuracy:", ((score + 0.0)/ len(preds)) * 100)
model.save("fruits" + ".h5")
print("Model saved")

Epoch 1/10
594/594 [==============================] - 58s 97ms/step - loss: 0.0532 - accuracy: 0.9822
Epoch 2/10
594/594 [==============================] - 61s 103ms/step - loss: 0.0464 - accuracy: 0.9836
Epoch 3/10
594/594 [==============================] - 61s 103ms/step - loss: 0.0395 - accuracy: 0.9863
Epoch 4/10
594/594 [==============================] - 58s 97ms/step - loss: 0.0379 - accuracy: 0.9864
Epoch 5/10
594/594 [==============================] - 57s 96ms/step - loss: 0.0335 - accuracy: 0.9890
Epoch 6/10
594/594 [==============================] - 57s 96ms/step - loss: 0.0333 - accuracy: 0.9881
Epoch 7/10
594/594 [==============================] - 56s 95ms/step - loss: 0.0309 - accuracy: 0.9890
Epoch 8/10
594/594 [==============================] - 60s 101ms/step - loss: 0.0291 - accuracy: 0.9891
Epoch 9/10
594/594 [==============================] - 56s 95ms/step - loss: 0.0236 - accuracy: 0.9909
Epoch 10/10
594/594 [==============================] - 57s 95ms/step - loss: 0.

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
from tensorflow.keras.models import load_model
path = "/content/drive/MyDrive/saved data"
model.save_weights(path, True)

In [ ]:
from keras.models import load_model
model.save('/content/drive/MyDrive/saved data')



ValueError: ignored